<a href="https://colab.research.google.com/github/sugban/SaturdaysAI/blob/main/Classification_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 5.0 MB/s 
     |████████████████████████████████| 3.0 MB 39.3 MB/s 
     |████████████████████████████████| 98 kB 6.5 MB/s 
     |████████████████████████████████| 46 kB 2.3 MB/s 
     |████████████████████████████████| 130 kB 22.7 MB/s 
     |████████████████████████████████| 99 kB 7.9 MB/s 
     |████████████████████████████████| 1.2 MB 37.1 MB/s 
     |████████████████████████████████| 8.8 MB 24.8 MB/s 
     |████████████████████████████████| 95 kB 3.9 MB/s 
     |████████████████████████████████| 187 kB 72.1 MB/s 
     |████████████████████████████████| 3.3 MB 38.0 MB/s 
     |████████████████████████████████| 1.3 MB 60.0 MB/s 
     |████████████████████████████████| 128 kB 56.2 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 342 kB 58.4 MB/s 
     |████████████████████████████████| 42

In [ ]:
import ee
import geemap

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [ ]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=SlR6oKYnvo0sNnFC08FnJwF3gSMKb_xKfduZBXsEZpo&tc=8sdnhb4j3brQ2KkZ9fS2SC2xHUQ5ttrpp1ighRO7BCs&cc=DOskiqHPuZipX6SlsuZYAZXV2qmAGXT4kvSIyMtf4ao

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhypzbk0qn9qFZZvzRlhRiKJmYwgfKpxH78n7TheBr19UaY6KMvqsI

Successfully saved authorization token.


In [ ]:
ee.Initialize()

In [ ]:
Map = geemap.Map()

In [ ]:
#ROI
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
roi = countries.filter(ee.Filter.eq("country_na","Denmark"))

In [ ]:
# # Define the urban location of interest as a point
lat = 55.676098 
lon = 12.568337
point = ee.Geometry.Point(lon, lat)

In [ ]:
Map = geemap.Map(center=(40, -100), zoom=4)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [ ]:
Map.addLayer(roi,{}, "Denmark",False);

In [ ]:
#Load Sentinel data

image = ee.ImageCollection("COPERNICUS/S2_SR").filterDate("2020-06-01","2020-06-30").filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",10)).filterBounds(point).median()

In [ ]:
visParamsTrue = {'bands': ["B4", "B3", "B2"], 'min': 0, 'max': 3, 'gamma': 1.1}
Map.addLayer(image.clip(point), visParamsTrue, "Sentinel 2020")
Map.centerObject(point, 10)

In [ ]:
# Load training points. The numeric property 'class' stores known labels.
training = ee.FeatureCollection('users/sugbansal/trainData_2020')
Map.addLayer(training, {}, 'Training data')

In [ ]:
# Create Training Data

label = "Class"
bands = ["B6","B7", "B8","B8A"];
input = image.select(bands)

In [ ]:
#Overlay the points on the imagery to get training (all of elements "water,forest,etc with bands )
trainImage = input.sampleRegions(**{
    'collection' : training, 
    'properties' : [label], 
    'scale': 30 
})

In [ ]:
#training and validation
trainingData  = trainImage.randomColumn()

In [ ]:
# separete data training test and trainig the model 

trainSet = trainingData.filter(ee.Filter.lessThan("random", 0.8))
testSet = trainingData.filter(ee.Filter.greaterThanOrEquals("random", 0.8))

In [ ]:
# Classification Model
classifier = ee.Classifier.smileRandomForest(5).train(trainSet, label, bands)

In [ ]:
# Classify the image(Sentinel data)
classified = input.classify(classifier)

In [ ]:
# Define a palette for the classification

landcoverPalette = [
                    "#0eb6fe", # water(0)
                    "#d63000", # urban (1)
                    "#2ea85c", # forest (2)
                    "#b4a899", # farmland (3)
]

In [ ]:
#Vizualization
Map.addLayer(
    classified.clip(roi), 
    {'palette': landcoverPalette, 'min':0, 'max': 3 }, 
    "Classification") #Model

In [ ]:
train_accuracy = classifier.confusionMatrix()
cf_matrix = train_accuracy.getInfo()
cf_matrix

[[208, 1, 0, 0], [0, 383, 0, 1], [0, 0, 78, 4], [0, 0, 1, 134]]

In [ ]:
train_accuracy.accuracy().getInfo()

0.991358024691358